In [2]:
%load_ext rpy2.ipython

In [3]:
%%R
library(DESeq2)
library(magrittr)
library(SummarizedExperiment)

start_time <- Sys.time()

IN_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_validation/input/"
OUT_DIR <- "/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/GEO_model_validation/output/"

GEO_model_validation_DATA_FIL <- "rse_gene_20.Rdata"

ensembl2rxns.df <- read.table(paste(IN_DIR,"Ensembl2ReactomeReactions.txt",sep=""),
                              sep="\t")

load(paste(IN_DIR,GEO_model_validation_DATA_FIL,sep=""))

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce, rownames, sapply, setdiff, sort,
    table, tapply, union, unique, unsplit, which.max, which.min


Attaching package: ‘S4Vectors’

The following object is masked from ‘package:utils’:

    findMatches

The following objects are masked from ‘package:base’:

    expand.grid, I, unname

Loading required package: IRanges
Loading required package: GenomicRanges
Loading required package: GenomeInfoDb
Loading require

In [4]:
%%R
tissue_data <- colData(rse_gene_20)
print(tissue_data)

DataFrame with 1581 rows and 203 columns
     external_id   rail_id       study sra.sample_acc.x sra.experiment_acc
     <character> <integer> <character>      <character>        <character>
1     SRR3087157   3025649   SRP068123       SRS1237142         SRX1518780
2     SRR1810239   1385373   SRP055141        SRS850421          SRX883048
3     SRR7790225   3294398   SRP159685       SRS3743326         SRX4645280
4     SRR7161462   3312721   SRP145605       SRS1264753         SRX4080490
5      DRR091804    176136   DRP003641        DRS045146          DRX085374
...          ...       ...         ...              ...                ...
1577  ERR2130627    831804   ERP104395       ERS1935420         ERX2187479
1578  SRR5277035   3409586   SRP100466       SRS1995094         SRX2580975
1579  SRR6884615   3850936   SRP136319       SRS3084220         SRX3836843
1580  SRR8857561   3845176   SRP190929       SRS4590819         SRX5645124
1581  ERR2588685    767021   ERP108893       ERS2492546    

...                                          ...
1577                                  1468402614
1578                                   440431928
1579                                  7408972546
1580                                  1235648851
1581                                  1877645506
     recount_qc.bc_auc.unique_reads_all_bases
                                    <numeric>
1                                  2687187323
2                                  5111211715
3                                  2867881166
4                                  1204592422
5                                  5761840315
...                                       ...
1577                               1126113651
1578                                390038157
1579                               6901919926
1580                               1061559007
1581                               1842327637
     recount_qc.bc_auc.unique_reads_annotated_bases recount_qc.bc_auc.all_%
                                

3                                              0
4                                              0
5                                              0
...                                          ...
1577                                           0
1578                                           0
1579                                           0
1580                                           0
1581                                           0
     recount_qc.star.%_of_reads_unmapped:_too_many_mismatches
                                                    <numeric>
1                                                           0
2                                                           0
3                                                           0
4                                                           0
5                                                           0
...                                                       ...
1577                                                        0
1

5                                         0
...                                     ...
1577                                      0
1578                                      0
1579                                      0
1580                                      0
1581                                      0
     recount_qc.star.number_of_reads_mapped_to_multiple_loci
                                                   <integer>
1                                                    4053053
2                                                    1253325
3                                                    1107671
4                                                    1208701
5                                                    2577014
...                                                      ...
1577                                                 6967599
1578                                                 1221111
1579                                                 1867134
1580                 

                                    <integer>
1                                        3244
2                                        4011
3                                        4412
4                                        1271
5                                        9476
...                                       ...
1577                                     1214
1578                                      425
1579                                    12790
1580                                      995
1581                                     3588
     recount_qc.star.number_of_splices:_at/ac2
                                     <integer>
1                                            0
2                                            0
3                                            0
4                                            0
5                                            0
...                                        ...
1577                                         0
1578                     

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [5]:
%%R
tissue_counts <- table(tissue_data$Major_tissue)

tissues_to_remove <- names(tissue_counts[tissue_counts <= 20])

rse_gene_20 <- rse_gene_20[, !(tissue_data$Major_tissue %in% c(tissues_to_remove, 'Spleen'))]

new_tissue_counts <- table(colData(rse_gene_20)$Major_tissue)
print(new_tissue_counts)

row_totals <- rowSums(assays(rse_gene_20)$raw_counts)

rse_gene_20 <- rse_gene_20[row_totals > 0, ]


  Adipose     Brain       Eye     Heart Intestine    Kidney     Liver      Lung 
      106       355        44        44       127        76       421        73 
      MOE    Muscle  Pancreas      Skin    Testes 
       26        68        48        27        30 


In [6]:
%%R
GEO_model_validation.cols <- rse_gene_20 %>% colData()
print(GEO_model_validation.cols$Major_tissue)

   [1] "Brain"     "Liver"     "Brain"     "Intestine" "Liver"     "Liver"    
   [7] "Liver"     "Liver"     "Eye"       "Heart"     "Brain"     "Intestine"
  [13] "Brain"     "Brain"     "MOE"       "Liver"     "Muscle"    "Liver"    
  [19] "Muscle"    "Lung"      "Heart"     "Liver"     "Liver"     "Lung"     
  [25] "Brain"     "Liver"     "Liver"     "Liver"     "Liver"     "Kidney"   
  [31] "Pancreas"  "Pancreas"  "Liver"     "Brain"     "Brain"     "Brain"    
  [37] "Muscle"    "Pancreas"  "Lung"      "Brain"     "Liver"     "Intestine"
  [43] "Liver"     "Brain"     "Muscle"    "Brain"     "Liver"     "Kidney"   
  [49] "Testes"    "Pancreas"  "Lung"      "Adipose"   "Eye"       "Liver"    
  [55] "Brain"     "Liver"     "Brain"     "Liver"     "Brain"     "Lung"     
  [61] "Brain"     "Adipose"   "Liver"     "Kidney"    "Liver"     "Intestine"
  [67] "Liver"     "Adipose"   "Testes"    "Brain"     "Liver"     "Adipose"  
  [73] "Liver"     "Adipose"   "Liver"     "Brain"  

[1237] "Pancreas"  "Brain"     "Kidney"    "Pancreas"  "Adipose"   "Liver"    
[1243] "Brain"     "Lung"      "Testes"    "Adipose"   "Liver"     "Muscle"   
[1249] "Liver"     "Liver"     "Brain"     "Brain"     "Liver"     "Kidney"   
[1255] "Intestine" "Testes"    "Muscle"    "Brain"     "Liver"     "Brain"    
[1261] "Brain"     "Adipose"   "Lung"      "Liver"     "Skin"      "Brain"    
[1267] "Liver"     "Pancreas"  "Lung"      "Liver"     "Eye"       "Adipose"  
[1273] "Intestine" "Liver"     "Intestine" "Pancreas"  "Pancreas"  "Intestine"
[1279] "Brain"     "Intestine" "Intestine" "Testes"    "Liver"     "Kidney"   
[1285] "Liver"     "Adipose"   "Liver"     "Liver"     "Liver"     "Intestine"
[1291] "Brain"     "Brain"     "Brain"     "Liver"     "Adipose"   "Brain"    
[1297] "Muscle"    "Brain"     "Liver"     "Kidney"    "Liver"     "Liver"    
[1303] "Liver"     "Liver"     "Liver"     "Adipose"   "Eye"       "Brain"    
[1309] "Intestine" "Brain"     "Liver"     "Skin"   

In [7]:
%%R
saveRDS(GEO_model_validation.cols$Major_tissue,file=paste(OUT_DIR,"GEO_model_validation_tissue_vec.Rds",sep=""))

In [8]:
%%R
saveRDS(GEO_model_validation.cols$external_id,file=paste(OUT_DIR,"GEO_model_validation_sample_detail_vec.Rds",sep=""))

In [9]:
%%R
GEO_model_validation_df_col_data <- colData(rse_gene_20)
head(GEO_model_validation_df_col_data$Row.names)
sample_id_list <- GEO_model_validation_df_col_data$external_id

In [10]:
%%R
print(sample_id_list)

   [1] "SRR3087157"  "SRR1810239"  "SRR7790225"  "SRR7161462"  "DRR179021"  
   [6] "SRR2033443"  "SRR8543821"  "SRR6680502"  "SRR5877191"  "SRR5009821" 
  [11] "SRR1821748"  "SRR7161319"  "SRR10400119" "SRR10399918" "ERR569315"  
  [16] "SRR1796662"  "SRR7138607"  "SRR2061987"  "SRR2033409"  "SRR5273652" 
  [21] "ERR2588669"  "SRR6194324"  "SRR6107152"  "SRR7511612"  "SRR10560410"
  [26] "SRR9029889"  "SRR6680465"  "SRR7641150"  "SRR6423120"  "SRR3161663" 
  [31] "SRR1510022"  "SRR7995510"  "SRR10181041" "SRR10467076" "SRR7948102" 
  [36] "SRR10400068" "SRR3355765"  "SRR3183046"  "SRR8616660"  "SRR6757012" 
  [41] "SRR1462368"  "SRR7161428"  "SRR6680425"  "SRR10399968" "SRR9026456" 
  [46] "SRR7765092"  "SRR9317373"  "SRR9062578"  "SRR1422067"  "SRR1510024" 
  [51] "SRR7138198"  "SRR7162053"  "SRR8690252"  "SRR3196800"  "SRR5083378" 
  [56] "SRR908296"   "SRR10399995" "SRR5275490"  "ERR1709716"  "SRR6495953" 
  [61] "SRR6907184"  "SRR3721870"  "SRR3197317"  "SRR3161689"  "SRR6680456" 

[1061] "SRR7161875"  "SRR8929805"  "SRR8315016"  "SRR6239818"  "SRR6357655" 
[1066] "SRR7161437"  "SRR5273671"  "ERR2362967"  "SRR1636593"  "SRR8141472" 
[1071] "DRR060872"   "SRR10523609" "SRR10399940" "SRR10399956" "SRR7162089" 
[1076] "ERR2220007"  "SRR1158580"  "SRR8285570"  "SRR10031017" "SRR1181750" 
[1081] "SRR6680408"  "SRR7161457"  "SRR9182325"  "SRR8858552"  "SRR543941"  
[1086] "SRR1805829"  "SRR3161707"  "SRR6680426"  "SRR10400092" "SRR9937476" 
[1091] "SRR1979371"  "SRR6194327"  "SRR6680376"  "SRR7765154"  "SRR7309705" 
[1096] "SRR10467114" "SRR10399938" "SRR2984141"  "SRR2153309"  "SRR10523600"
[1101] "SRR5461488"  "SRR4296300"  "ERR1121525"  "SRR5928844"  "SRR4317608" 
[1106] "ERR1624151"  "SRR5464217"  "SRR7890188"  "SRR449412"   "SRR6435602" 
[1111] "SRR3087170"  "SRR6122005"  "SRR10399937" "SRR6653183"  "SRR7292542" 
[1116] "SRR7229187"  "SRR6973974"  "SRR9026465"  "SRR7467395"  "SRR3126189" 
[1121] "SRR6225335"  "SRR10400168" "SRR6339685"  "SRR3087148"  "SRR9008321" 

In [11]:
%%R
GEO_model_validation.df <- rse_gene_20 %>% SummarizedExperiment::assay() %>% as.data.frame()
print(GEO_model_validation.df)

                        V1     V2     V3    V4    V5     V6    V7    V8    V9
ENSMUSG00000079800.2     0    763      0    36     0    100     0     0     0
ENSMUSG00000095092.1     0    137      0     0     0     50     0     0     0
ENSMUSG00000079192.2     0    932      0     0     0      0     0     0     0
ENSMUSG00000079794.2     0    133      0   195     0    150     9     0     0
ENSMUSG00000094799.1     0      0      0   121     0      0     0     0     0
ENSMUSG00000095250.1     0      0      0   132     0    497     0     0     0
ENSMUSG00000095787.1     0    470      0     0     0      0     0     0     0
ENSMUSG00000096100.1     0      0      0     0     5     50     0     0     0
ENSMUSG00000094054.1    75   2036      0   104     0    100     0   698     0
ENSMUSG00000095672.1     0      0      0   101     0      0     0     0     0
ENSMUSG00000079190.3     0   1620      0     0     0    611     0   198     0
ENSMUSG00000094514.1     0      0      0     0     0   1282     

ENSMUSG00000094303.1     0     0     0      0      0     0      0     0      0
ENSMUSG00000096236.1     0     0     0      0      0     0      0     0      0
ENSMUSG00000096756.1     0     0     0      0      0     0      0     0      0
                       V29  V30  V31    V32   V33   V34   V35   V36   V37   V38
ENSMUSG00000079800.2     0    0    0      0     0     0   100  8487     0     0
ENSMUSG00000095092.1     0    0    0      0     0     0     0   141     0    42
ENSMUSG00000079192.2     0    0    0      0    91     0     0  3978     0     0
ENSMUSG00000079794.2     0    0    0      0     0     0     0   836     0     0
ENSMUSG00000094799.1     0    0    0      0     0     0     0   100     0     0
ENSMUSG00000095250.1     0   72    0    164     0   200    99   797     0    84
ENSMUSG00000095787.1     0    0    0      0     0   100     0  3119     0     0
ENSMUSG00000096100.1     0    0    0      0    44     4     0  5572     0     0
ENSMUSG00000094054.1     0    0    0      0

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



  25334 266749 130227 20600
ENSMUSG00000089163.1     0     0     0      0     0      0      0      0     0
ENSMUSG00000061654.3 25956 11796     0   6276     0    510  41435  37168     0
ENSMUSG00000079834.2 11279 35452 15404 117749 15375  15979  77319  39007  5922
ENSMUSG00000099278.1     0     0     0      0     0      0      0      0     0
ENSMUSG00000094121.1     0     0     0     90     0      0      0      0     0
ENSMUSG00000094576.1     0     0     0    169     0      0    129    390     0
ENSMUSG00000079764.1   139     0     0      0     0      0      0      0     0
ENSMUSG00000079774.2     0     0     0      0     0      0      0      0     0
ENSMUSG00000079773.2 10757   113     0   1364    61    653   3076  13115     0
ENSMUSG00000096271.1  8957   516   916  46017  1343 181692   2908  55875   350
ENSMUSG00000094741.1     0     0     0      0     0      0      0      0     0
ENSMUSG00000074720.1     0     0     0      0     0      0      0      0     0
ENSMUSG00000094337.7    

ENSMUSG00000089163.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000061654.3     0     0  45114  44742   300   939    75   914   102
ENSMUSG00000079834.2  5516 11102  26223  13552  7502 28301  8419  9120 11469
ENSMUSG00000099278.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000094121.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000094576.1     0     0      0      0     0     0     0    51     0
ENSMUSG00000079764.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000079774.2     0     0      0      0     0     0     0     0     0
ENSMUSG00000079773.2    50     2  14696   2011   488    82    75   283     0
ENSMUSG00000096271.1   650    47  31000    112 48250 30437  5489 49687     0
ENSMUSG00000094741.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000074720.1     0     0      0      0     0     0     0     0     0
ENSMUSG00000094337.7     0     0      0      0     0     0     0     0     0

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



      0   100     0      0     0    50
ENSMUSG00000095207.1     0     0     0      0     0     0      0     0     0
ENSMUSG00000093828.7     0     0     0      0     0   225      0     0     0
ENSMUSG00000095623.1     0     0     0      0     0     0      0     0     0
ENSMUSG00000094661.1     0     0   146     73   236     0    140   799   349
ENSMUSG00000095320.1     0     0  2710    400  1481    86    527 28618 10775
ENSMUSG00000094350.1     0     0     0      0     0     0      0    85    51
ENSMUSG00000096237.1   608     0     0    171   424   620    392   568   457
ENSMUSG00000095742.1  1102     0  3739   1868  2816     0   5159 44177  5897
ENSMUSG00000095666.1     0     0     0      0     0    74      0     0     0
ENSMUSG00000096680.1     0     0     0      0     0     0      0     0     0
ENSMUSG00000095505.1     0     0     0      0    67     0      0     0     0
ENSMUSG00000094303.1     0     0     0      0     0   300      0     0     0
ENSMUSG00000096236.1     0     0     

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     0     0     0     0     0      0     0      0     0
ENSMUSG00000095623.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000094661.1    39     0     0     0     0    480     0    625     0
ENSMUSG00000095320.1 18328   502  6182  1023   190  78417   306    623   100
ENSMUSG00000094350.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000096237.1  1941     0   299   722     0      0  3888    101   212
ENSMUSG00000095742.1 22938 25708 17765    32  2915  51512 18470   8215   362
ENSMUSG00000095666.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000096680.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000095505.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000094303.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000096236.1     0     0     0     0     0      0     0      0     0
ENSMUSG00000096756.1     0     0     0     0     0      0     0      0     0
                   

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [12]:
%%R
ensembl_wo_ids <- gsub("\\.[0-9]+","",rownames(GEO_model_validation.df))
print(ensembl_wo_ids)

 "ENSMUSG00000098499" "ENSMUSG00000065273"
[21238] "ENSMUSG00000098269" "ENSMUSG00000019432" "ENSMUSG00000073411"
[21241] "ENSMUSG00000092444" "ENSMUSG00000092438" "ENSMUSG00000079507"
[21244] "ENSMUSG00000091655" "ENSMUSG00000091705" "ENSMUSG00000092253"
[21247] "ENSMUSG00000079505" "ENSMUSG00000035929" "ENSMUSG00000055413"
[21250] "ENSMUSG00000092560" "ENSMUSG00000073409" "ENSMUSG00000060550"
[21253] "ENSMUSG00000092278" "ENSMUSG00000067235" "ENSMUSG00000093516"
[21256] "ENSMUSG00000024406" "ENSMUSG00000050410" "ENSMUSG00000040312"
[21259] "ENSMUSG00000024409" "ENSMUSG00000039518" "ENSMUSG00000092409"
[21262] "ENSMUSG00000039269" "ENSMUSG00000090588" "ENSMUSG00000073408"
[21265] "ENSMUSG00000090509" "ENSMUSG00000092389" "ENSMUSG00000038838"
[21268] "ENSMUSG00000001524" "ENSMUSG00000093921" "ENSMUSG00000003534"
[21271] "ENSMUSG00000086867" "ENSMUSG00000092219" "ENSMUSG00000092600"
[21274] "ENSMUSG00000092171" "ENSMUSG00000003541" "ENSMUSG00000059714"
[21277] "ENSMUSG00000001525" "ENSM

[23116] "ENSMUSG00000073569" "ENSMUSG00000117844" "ENSMUSG00000118074"
[23119] "ENSMUSG00000054477" "ENSMUSG00000118037" "ENSMUSG00000088649"
[23122] "ENSMUSG00000098284" "ENSMUSG00000104009" "ENSMUSG00000103616"
[23125] "ENSMUSG00000033949" "ENSMUSG00000024477" "ENSMUSG00000071855"
[23128] "ENSMUSG00000117742" "ENSMUSG00000118282" "ENSMUSG00000095541"
[23131] "ENSMUSG00000118002" "ENSMUSG00000046487" "ENSMUSG00000088804"
[23134] "ENSMUSG00000118116" "ENSMUSG00000033319" "ENSMUSG00000056130"
[23137] "ENSMUSG00000033184" "ENSMUSG00000057561" "ENSMUSG00000117780"
[23140] "ENSMUSG00000033022" "ENSMUSG00000032905" "ENSMUSG00000117429"
[23143] "ENSMUSG00000117599" "ENSMUSG00000117372" "ENSMUSG00000024480"
[23146] "ENSMUSG00000094292" "ENSMUSG00000024481" "ENSMUSG00000117630"
[23149] "ENSMUSG00000073568" "ENSMUSG00000117411" "ENSMUSG00000117588"
[23152] "ENSMUSG00000087857" "ENSMUSG00000117621" "ENSMUSG00000117377"
[23155] "ENSMUSG00000093158" "ENSMUSG00000117350" "ENSMUSG00000042705"
[23158

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[26479] "ENSMUSG00000104262" "ENSMUSG00000103546" "ENSMUSG00000102662"
[26482] "ENSMUSG00000027035" "ENSMUSG00000034738" "ENSMUSG00000005233"
[26485] "ENSMUSG00000005232" "ENSMUSG00000086819" "ENSMUSG00000083592"
[26488] "ENSMUSG00000027048" "ENSMUSG00000027068" "ENSMUSG00000027070"
[26491] "ENSMUSG00000063145" "ENSMUSG00000075307" "ENSMUSG00000027086"
[26494] "ENSMUSG00000082345" "ENSMUSG00000042133" "ENSMUSG00000083007"
[26497] "ENSMUSG00000070883" "ENSMUSG00000081857" "ENSMUSG00000027088"
[26500] "ENSMUSG00000042155" "ENSMUSG00000068882" "ENSMUSG00000089902"
[26503] "ENSMUSG00000051730" "ENSMUSG00000085735" "ENSMUSG00000044308"
[26506] "ENSMUSG00000086132" "ENSMUSG00000042064" "ENSMUSG00000047965"
[26509] "ENSMUSG00000075304" "ENSMUSG00000086515" "ENSMUSG00000104076"
[26512] "ENSMUSG00000075302" "ENSMUSG00000087264" "ENSMUSG00000070880"
[26515] "ENSMUSG00000086699" "ENSMUSG00000097025" "ENSMUSG00000014959"
[26518] "ENSMUSG00000041997" "ENSMUSG00000082303" "ENSMUSG00000085868"
[26521

[28198] "ENSMUSG00000086800" "ENSMUSG00000083088" "ENSMUSG00000087033"
[28201] "ENSMUSG00000086620" "ENSMUSG00000032852" "ENSMUSG00000027460"
[28204] "ENSMUSG00000087004" "ENSMUSG00000088491" "ENSMUSG00000027459"
[28207] "ENSMUSG00000027463" "ENSMUSG00000085946" "ENSMUSG00000060257"
[28210] "ENSMUSG00000032802" "ENSMUSG00000068079" "ENSMUSG00000081087"
[28213] "ENSMUSG00000074698" "ENSMUSG00000027465" "ENSMUSG00000027466"
[28216] "ENSMUSG00000032715" "ENSMUSG00000081434" "ENSMUSG00000068062"
[28219] "ENSMUSG00000059361" "ENSMUSG00000051817" "ENSMUSG00000074682"
[28222] "ENSMUSG00000088139" "ENSMUSG00000032680" "ENSMUSG00000083230"
[28225] "ENSMUSG00000074681" "ENSMUSG00000082689" "ENSMUSG00000049560"
[28228] "ENSMUSG00000027468" "ENSMUSG00000074680" "ENSMUSG00000074679"
[28231] "ENSMUSG00000044249" "ENSMUSG00000084935" "ENSMUSG00000090485"
[28234] "ENSMUSG00000056544" "ENSMUSG00000050645" "ENSMUSG00000062124"
[28237] "ENSMUSG00000089684" "ENSMUSG00000044863" "ENSMUSG00000074678"
[28240

...

[32176] "ENSMUSG00000087180" "ENSMUSG00000093947" "ENSMUSG00000082949"
[32179] "ENSMUSG00000085522" "ENSMUSG00000065053" "ENSMUSG00000082254"
[32182] "ENSMUSG00000084201" "ENSMUSG00000084440" "ENSMUSG00000028284"
[32185] "ENSMUSG00000082398" "ENSMUSG00000083872" "ENSMUSG00000083564"
[32188] "ENSMUSG00000087049" "ENSMUSG00000084611" "ENSMUSG00000086150"
[32191] "ENSMUSG00000040270" "ENSMUSG00000051056" "ENSMUSG00000081879"
[32194] "ENSMUSG00000028282" "ENSMUSG00000081592" "ENSMUSG00000058006"
[32197] "ENSMUSG00000045854" "ENSMUSG00000083565" "ENSMUSG00000040183"
[32200] "ENSMUSG00000094344" "ENSMUSG00000081025" "ENSMUSG00000028278"
[32203] "ENSMUSG00000085037" "ENSMUSG00000028277" "ENSMUSG00000082851"
[32206] "ENSMUSG00000023267" "ENSMUSG00000028280" "ENSMUSG00000054659"
[32209] "ENSMUSG00000083887" "ENSMUSG00000054679" "ENSMUSG00000040128"
[32212] "ENSMUSG00000086127" "ENSMUSG00000098310" "ENSMUSG00000081206"
[32215] "ENSMUSG00000028274" "ENSMUSG00000096184" "ENSMUSG00000077397"
[32218

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000087171" "ENSMUSG00000086856" "ENSMUSG00000028909"
[33946] "ENSMUSG00000028910" "ENSMUSG00000102602" "ENSMUSG00000028911"
[33949] "ENSMUSG00000083013" "ENSMUSG00000085667" "ENSMUSG00000118394"
[33952] "ENSMUSG00000070720" "ENSMUSG00000098577" "ENSMUSG00000084869"
[33955] "ENSMUSG00000028906" "ENSMUSG00000070717" "ENSMUSG00000083368"
[33958] "ENSMUSG00000102796" "ENSMUSG00000050511" "ENSMUSG00000081394"
[33961] "ENSMUSG00000040025" "ENSMUSG00000080597" "ENSMUSG00000083757"
[33964] "ENSMUSG00000084105" "ENSMUSG00000028901" "ENSMUSG00000100813"
[33967] "ENSMUSG00000077323" "ENSMUSG00000099937" "ENSMUSG00000028899"
[33970] "ENSMUSG00000089687" "ENSMUSG00000065097" "ENSMUSG00000105911"
[33973] "ENSMUSG00000064604" "ENSMUSG00000064949" "ENSMUSG00000080615"
[33976] "ENSMUSG00000086290" "ENSMUSG00000028898" "ENSMUSG00000065353"
[33979] "ENSMUSG00000064387" "ENSMUSG00000091021" "ENSMUSG00000028896"
[33982] "ENSMUSG00000085241" "ENSMUSG00000096350" "ENSMUSG00000066043"
[33985] "ENSM

[34633] "ENSMUSG00000082073" "ENSMUSG00000087071" "ENSMUSG00000088774"
[34636] "ENSMUSG00000029026" "ENSMUSG00000029030" "ENSMUSG00000029029"
[34639] "ENSMUSG00000081276" "ENSMUSG00000057751" "ENSMUSG00000029032"
[34642] "ENSMUSG00000097871" "ENSMUSG00000086565" "ENSMUSG00000084987"
[34645] "ENSMUSG00000039410" "ENSMUSG00000085069" "ENSMUSG00000098424"
[34648] "ENSMUSG00000086810" "ENSMUSG00000051276" "ENSMUSG00000046637"
[34651] "ENSMUSG00000087437" "ENSMUSG00000058183" "ENSMUSG00000029059"
[34654] "ENSMUSG00000042333" "ENSMUSG00000092483" "ENSMUSG00000048001"
[34657] "ENSMUSG00000029056" "ENSMUSG00000073686" "ENSMUSG00000029055"
[34660] "ENSMUSG00000029047" "ENSMUSG00000029048" "ENSMUSG00000029049"
[34663] "ENSMUSG00000029050" "ENSMUSG00000073684" "ENSMUSG00000098729"
[34666] "ENSMUSG00000029053" "ENSMUSG00000029054" "ENSMUSG00000084803"
[34669] "ENSMUSG00000078490" "ENSMUSG00000023153" "ENSMUSG00000082429"
[34672] "ENSMUSG00000029064" "ENSMUSG00000029063" "ENSMUSG00000085063"
[34675

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [13]:
%%R
rownames(GEO_model_validation.df) <- ensembl_wo_ids
print(rownames(GEO_model_validation.df))

 "ENSMUSG00000093228" "ENSMUSG00000099658"
 [1126] "ENSMUSG00000025958" "ENSMUSG00000025956" "ENSMUSG00000101832"
 [1129] "ENSMUSG00000100437" "ENSMUSG00000087213" "ENSMUSG00000070871"
 [1132] "ENSMUSG00000045005" "ENSMUSG00000102140" "ENSMUSG00000097264"
 [1135] "ENSMUSG00000051344" "ENSMUSG00000084416" "ENSMUSG00000077471"
 [1138] "ENSMUSG00000090042" "ENSMUSG00000025955" "ENSMUSG00000070870"
 [1141] "ENSMUSG00000067299" "ENSMUSG00000025952" "ENSMUSG00000073658"
 [1144] "ENSMUSG00000100774" "ENSMUSG00000044429" "ENSMUSG00000100846"
 [1147] "ENSMUSG00000044816" "ENSMUSG00000025950" "ENSMUSG00000025949"
 [1150] "ENSMUSG00000025946" "ENSMUSG00000100503" "ENSMUSG00000100504"
 [1153] "ENSMUSG00000099383" "ENSMUSG00000100340" "ENSMUSG00000100439"
 [1156] "ENSMUSG00000099819" "ENSMUSG00000082565" "ENSMUSG00000082774"
 [1159] "ENSMUSG00000025945" "ENSMUSG00000081623" "ENSMUSG00000083242"
 [1162] "ENSMUSG00000083289" "ENSMUSG00000088850" "ENSMUSG00000073656"
 [1165] "ENSMUSG00000065694"
 [347

 [4123] "ENSMUSG00000071335" "ENSMUSG00000038528" "ENSMUSG00000111608"
 [4126] "ENSMUSG00000096171" "ENSMUSG00000038522" "ENSMUSG00000077200"
 [4129] "ENSMUSG00000019838" "ENSMUSG00000110781" "ENSMUSG00000111045"
 [4132] "ENSMUSG00000102632" "ENSMUSG00000038510" "ENSMUSG00000019837"
 [4135] "ENSMUSG00000106147" "ENSMUSG00000065870" "ENSMUSG00000111665"
 [4138] "ENSMUSG00000111213" "ENSMUSG00000075232" "ENSMUSG00000111788"
 [4141] "ENSMUSG00000038491" "ENSMUSG00000038481" "ENSMUSG00000111251"
 [4144] "ENSMUSG00000111108" "ENSMUSG00000019834" "ENSMUSG00000063428"
 [4147] "ENSMUSG00000045555" "ENSMUSG00000089186" "ENSMUSG00000038446"
 [4150] "ENSMUSG00000019831" "ENSMUSG00000111098" "ENSMUSG00000110889"
 [4153] "ENSMUSG00000046922" "ENSMUSG00000111576" "ENSMUSG00000111587"
 [4156] "ENSMUSG00000093933" "ENSMUSG00000038417" "ENSMUSG00000110822"
 [4159] "ENSMUSG00000111601" "ENSMUSG00000094663" "ENSMUSG00000111315"
 [4162] "ENSMUSG00000098818" "ENSMUSG00000091415" "ENSMUSG00000019826"
 [4165

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 [5842] "ENSMUSG00000118419" "ENSMUSG00000020115" "ENSMUSG00000092841"
 [5845] "ENSMUSG00000034667" "ENSMUSG00000047642" "ENSMUSG00000112526"
 [5848] "ENSMUSG00000053684" "ENSMUSG00000112596" "ENSMUSG00000020121"
 [5851] "ENSMUSG00000112184" "ENSMUSG00000111973" "ENSMUSG00000112815"
 [5854] "ENSMUSG00000034620" "ENSMUSG00000111952" "ENSMUSG00000098138"
 [5857] "ENSMUSG00000112305" "ENSMUSG00000112774" "ENSMUSG00000020123"
 [5860] "ENSMUSG00000112794" "ENSMUSG00000112873" "ENSMUSG00000112095"
 [5863] "ENSMUSG00000098522" "ENSMUSG00000099225" "ENSMUSG00000112417"
 [5866] "ENSMUSG00000111957" "ENSMUSG00000112220" "ENSMUSG00000034613"
 [5869] "ENSMUSG00000065406" "ENSMUSG00000112237" "ENSMUSG00000034602"
 [5872] "ENSMUSG00000112879" "ENSMUSG00000099088" "ENSMUSG00000020124"
 [5875] "ENSMUSG00000112294" "ENSMUSG00000084461" "ENSMUSG00000112036"
 [5878] "ENSMUSG00000111945" "ENSMUSG00000044071" "ENSMUSG00000112603"
 [5881] "ENSMUSG00000112250" "ENSMUSG00000089619" "ENSMUSG00000112364"
 [5884

 [6532] "ENSMUSG00000088438" "ENSMUSG00000080735" "ENSMUSG00000081658"
 [6535] "ENSMUSG00000082243" "ENSMUSG00000084064" "ENSMUSG00000083201"
 [6538] "ENSMUSG00000032889" "ENSMUSG00000084905" "ENSMUSG00000032878"
 [6541] "ENSMUSG00000084966" "ENSMUSG00000106400" "ENSMUSG00000076318"
 [6544] "ENSMUSG00000065539" "ENSMUSG00000065415" "ENSMUSG00000085510"
 [6547] "ENSMUSG00000020467" "ENSMUSG00000081695" "ENSMUSG00000020464"
 [6550] "ENSMUSG00000086725" "ENSMUSG00000020463" "ENSMUSG00000020462"
 [6553] "ENSMUSG00000083298" "ENSMUSG00000080517" "ENSMUSG00000080827"
 [6556] "ENSMUSG00000082721" "ENSMUSG00000032740" "ENSMUSG00000032673"
 [6559] "ENSMUSG00000085165" "ENSMUSG00000020459" "ENSMUSG00000020460"
 [6562] "ENSMUSG00000081575" "ENSMUSG00000020461" "ENSMUSG00000086707"
 [6565] "ENSMUSG00000085893" "ENSMUSG00000084850" "ENSMUSG00000083917"
 [6568] "ENSMUSG00000020458" "ENSMUSG00000078157" "ENSMUSG00000080851"
 [6571] "ENSMUSG00000088135" "ENSMUSG00000080906" "ENSMUSG00000044072"
 [6574

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSMUSG00000105323" "ENSMUSG00000094262" "ENSMUSG00000106216"
[39274] "ENSMUSG00000076552" "ENSMUSG00000094006" "ENSMUSG00000104679"
[39277] "ENSMUSG00000095633" "ENSMUSG00000104690" "ENSMUSG00000092746"
[39280] "ENSMUSG00000104551" "ENSMUSG00000076555" "ENSMUSG00000105369"
[39283] "ENSMUSG00000076556" "ENSMUSG00000106546" "ENSMUSG00000106016"
[39286] "ENSMUSG00000106213" "ENSMUSG00000096833" "ENSMUSG00000105301"
[39289] "ENSMUSG00000094319" "ENSMUSG00000095753" "ENSMUSG00000079542"
[39292] "ENSMUSG00000094993" "ENSMUSG00000106298" "ENSMUSG00000076562"
[39295] "ENSMUSG00000105725" "ENSMUSG00000076563" "ENSMUSG00000106256"
[39298] "ENSMUSG00000076564" "ENSMUSG00000094094" "ENSMUSG00000096422"
[39301] "ENSMUSG00000094433" "ENSMUSG00000105915" "ENSMUSG00000095007"
[39304] "ENSMUSG00000104533" "ENSMUSG00000105955" "ENSMUSG00000076569"
[39307] "ENSMUSG00000105451" "ENSMUSG00000076570" "ENSMUSG00000076571"
[39310] "ENSMUSG00000076572" "ENSMUSG00000076573" "ENSMUSG00000094641"
[39313] "ENSM

[39961] "ENSMUSG00000083086" "ENSMUSG00000035158" "ENSMUSG00000107902"
[39964] "ENSMUSG00000090667" "ENSMUSG00000107929" "ENSMUSG00000107614"
[39967] "ENSMUSG00000107938" "ENSMUSG00000107514" "ENSMUSG00000105902"
[39970] "ENSMUSG00000093559" "ENSMUSG00000088699" "ENSMUSG00000030067"
[39973] "ENSMUSG00000084547" "ENSMUSG00000107583" "ENSMUSG00000093661"
[39976] "ENSMUSG00000030068" "ENSMUSG00000072875" "ENSMUSG00000030069"
[39979] "ENSMUSG00000097548" "ENSMUSG00000108082" "ENSMUSG00000107713"
[39982] "ENSMUSG00000084319" "ENSMUSG00000087946" "ENSMUSG00000107204"
[39985] "ENSMUSG00000107528" "ENSMUSG00000108151" "ENSMUSG00000107558"
[39988] "ENSMUSG00000108269" "ENSMUSG00000107461" "ENSMUSG00000107690"
[39991] "ENSMUSG00000072872" "ENSMUSG00000087929" "ENSMUSG00000107663"
[39994] "ENSMUSG00000101553" "ENSMUSG00000079224" "ENSMUSG00000108019"
[39997] "ENSMUSG00000108122" "ENSMUSG00000035378" "ENSMUSG00000084157"
[40000] "ENSMUSG00000107989" "ENSMUSG00000030074" "ENSMUSG00000052144"
[40003

[40651] "ENSMUSG00000030167" "ENSMUSG00000083848" "ENSMUSG00000076212"
[40654] "ENSMUSG00000107715" "ENSMUSG00000067610" "ENSMUSG00000043932"
[40657] "ENSMUSG00000071158" "ENSMUSG00000082069" "ENSMUSG00000108235"
[40660] "ENSMUSG00000092047" "ENSMUSG00000014543" "ENSMUSG00000096482"
[40663] "ENSMUSG00000030173" "ENSMUSG00000083120" "ENSMUSG00000088426"
[40666] "ENSMUSG00000061769" "ENSMUSG00000079852" "ENSMUSG00000089383"
[40669] "ENSMUSG00000108099" "ENSMUSG00000072717" "ENSMUSG00000094188"
[40672] "ENSMUSG00000089727" "ENSMUSG00000096331" "ENSMUSG00000072721"
[40675] "ENSMUSG00000094579" "ENSMUSG00000033024" "ENSMUSG00000067599"
[40678] "ENSMUSG00000072718" "ENSMUSG00000030178" "ENSMUSG00000094408"
[40681] "ENSMUSG00000067591" "ENSMUSG00000079853" "ENSMUSG00000082777"
[40684] "ENSMUSG00000084204" "ENSMUSG00000107492" "ENSMUSG00000098212"
[40687] "ENSMUSG00000098025" "ENSMUSG00000107787" "ENSMUSG00000098115"
[40690] "ENSMUSG00000107519" "ENSMUSG00000098084" "ENSMUSG00000098214"
[40693

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
%%R
print(ensembl2rxns.df$V1)

 "DDB_G0292244"         "DDB_G0292244"        
 [8011] "DDB_G0292244"         "DDB_G0292244"         "DDB_G0292244"        
 [8014] "DDB_G0292244"         "DDB_G0292244"         "DDB_G0292244"        
 [8017] "DDB_G0292244"         "DDB_G0292244"         "DDB_G0292724"        
 [8020] "DDB_G0292724"         "DDB_G0292724"         "DDB_G0292724"        
 [8023] "DDB_G0292724"         "DDB_G0292730"         "DDB_G0292730"        
 [8026] "DDB_G0292736"         "DDB_G0292736"         "DDB_G0292736"        
 [8029] "DDB_G0292736"         "DDB_G0292736"         "DDB_G0292736"        
 [8032] "DDB_G0292736"         "DDB_G0292736"         "DDB_G0292736"        
 [8035] "DDB_G0292736"         "DDB_G0292736"         "DDB_G0292736"        
 [8038] "DDB_G0292736"         "DDB_G0292736"         "DDB_G0292736"        
 [8041] "DDB_G0292760"         "DDB_G0292770"         "DDB_G0292770"        
 [8044] "DDB_G0292778"         "DDB_G0292784"         "DDB_G0292784"        
 [8047] "DDB_G0292784"       

 [9853] "ENSBTAG00000000570.6" "ENSBTAG00000000570.6" "ENSBTAG00000000570.6"
 [9856] "ENSBTAG00000000570.6" "ENSBTAG00000000570.6" "ENSBTAG00000000570.6"
 [9859] "ENSBTAG00000000570.6" "ENSBTAG00000000570.6" "ENSBTAG00000000570.6"
 [9862] "ENSBTAG00000000570.6" "ENSBTAG00000000570.6" "ENSBTAG00000000570.6"
 [9865] "ENSBTAG00000000570.6" "ENSBTAG00000000570.6" "ENSBTAG00000000570.6"
 [9868] "ENSBTAG00000000570.6" "ENSBTAG00000000571"   "ENSBTAG00000000571"  
 [9871] "ENSBTAG00000000571.6" "ENSBTAG00000000571.6" "ENSBTAG00000000573"  
 [9874] "ENSBTAG00000000573"   "ENSBTAG00000000573"   "ENSBTAG00000000573"  
 [9877] "ENSBTAG00000000573"   "ENSBTAG00000000573"   "ENSBTAG00000000573"  
 [9880] "ENSBTAG00000000573"   "ENSBTAG00000000573"   "ENSBTAG00000000573"  
 [9883] "ENSBTAG00000000573"   "ENSBTAG00000000573"   "ENSBTAG00000000573.6"
 [9886] "ENSBTAG00000000573.6" "ENSBTAG00000000573.6" "ENSBTAG00000000573.6"
 [9889] "ENSBTAG00000000573.6" "ENSBTAG00000000573.6" "ENSBTAG00000000573.6"

[10489] "ENSBTAG00000000752.6" "ENSBTAG00000000752.6" "ENSBTAG00000000752.6"
[10492] "ENSBTAG00000000752.6" "ENSBTAG00000000752.6" "ENSBTAG00000000752.6"
[10495] "ENSBTAG00000000752.6" "ENSBTAG00000000752.6" "ENSBTAG00000000752.6"
[10498] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10501] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10504] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10507] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10510] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10513] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10516] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10519] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10522] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  
[10525] "ENSBTAG00000000753"   "ENSBTAG00000000753"   "ENSBTAG00000000753"  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 "ENSBTAG00000002350"  
[13189] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13192] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13195] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13198] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13201] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13204] "ENSBTAG00000002350"   "ENSBTAG00000002350"   "ENSBTAG00000002350"  
[13207] "ENSBTAG00000002350"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13210] "ENSBTAG00000002357"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13213] "ENSBTAG00000002357"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13216] "ENSBTAG00000002357"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13219] "ENSBTAG00000002357"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13222] "ENSBTAG00000002357"   "ENSBTAG00000002357"   "ENSBTAG00000002357"  
[13225] "ENSBTAG00000002357"   "ENSBTAG00000002357" 

...

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[84574] "ENSBTAP00000054212"   "ENSBTAP00000054212"   "ENSBTAP00000054213"  
[84577] "ENSBTAP00000054213"   "ENSBTAP00000054214"   "ENSBTAP00000054214"  
[84580] "ENSBTAP00000054219"   "ENSBTAP00000054247"   "ENSBTAP00000054254"  
[84583] "ENSBTAP00000054254"   "ENSBTAP00000054254"   "ENSBTAP00000054254"  
[84586] "ENSBTAP00000054254"   "ENSBTAP00000054263"   "ENSBTAP00000054275"  
[84589] "ENSBTAP00000054275"   "ENSBTAP00000054275"   "ENSBTAP00000054275"  
[84592] "ENSBTAP00000054275"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84595] "ENSBTAP00000054276"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84598] "ENSBTAP00000054276"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84601] "ENSBTAP00000054276"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84604] "ENSBTAP00000054276"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84607] "ENSBTAP00000054276"   "ENSBTAP00000054276"   "ENSBTAP00000054276"  
[84610] "ENSBTAP00000054276"   "ENSBTAP00000054282"   "ENSBTAP00000054283"  

[85210] "ENSBTAP00000055280"   "ENSBTAP00000055281"   "ENSBTAP00000055281"  
[85213] "ENSBTAP00000055281"   "ENSBTAP00000055281"   "ENSBTAP00000055281"  
[85216] "ENSBTAP00000055281"   "ENSBTAP00000055281"   "ENSBTAP00000055281"  
[85219] "ENSBTAP00000055281"   "ENSBTAP00000055281"   "ENSBTAP00000055281"  
[85222] "ENSBTAP00000055281"   "ENSBTAP00000055281"   "ENSBTAP00000055281"  
[85225] "ENSBTAP00000055281"   "ENSBTAP00000055288"   "ENSBTAP00000055298"  
[85228] "ENSBTAP00000055298"   "ENSBTAP00000055298"   "ENSBTAP00000055298"  
[85231] "ENSBTAP00000055299"   "ENSBTAP00000055299"   "ENSBTAP00000055314"  
[85234] "ENSBTAP00000055325"   "ENSBTAP00000055348"   "ENSBTAP00000055348"  
[85237] "ENSBTAP00000055348"   "ENSBTAP00000055348"   "ENSBTAP00000055353"  
[85240] "ENSBTAP00000055353"   "ENSBTAP00000055353"   "ENSBTAP00000055353"  
[85243] "ENSBTAP00000055353"   "ENSBTAP00000055353"   "ENSBTAP00000055353"  
[85246] "ENSBTAP00000055353"   "ENSBTAP00000055364"   "ENSBTAP00000055370"  

[96898] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96901] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96904] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96907] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96910] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96913] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96916] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96919] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96922] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96925] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001948"  
[96928] "ENSBTAT00000001948"   "ENSBTAT00000001948"   "ENSBTAT00000001950"  
[96931] "ENSBTAT00000001953"   "ENSBTAT00000001953"   "ENSBTAT00000001970"  
[96934] "ENSBTAT00000001973"   "ENSBTAT00000001976"   "ENSBTAT00000001976"  

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [15]:
%%R
reactome_ensembl_ids <- intersect(ensembl2rxns.df$V1,ensembl_wo_ids)
print(length(reactome_ensembl_ids))

[1] 4759


In [16]:
%%R
saveRDS(reactome_ensembl_ids,file=paste(OUT_DIR,"reactome_ensembl_ids.Rds",sep=""))

In [17]:
%%R
GEO_model_validation.df <- GEO_model_validation.df[reactome_ensembl_ids,]
length(GEO_model_validation.df)

[1] 1445


In [18]:
%%R
saveRDS(GEO_model_validation.df,file=paste(OUT_DIR,"GEO_model_validation_df.Rds",sep=""))